In [ ]:
# 1. 라이브러리 임포트 및 API 키 설정
# ===================================================================
import os
import urllib.request
import datetime
import json
import time
import ssl # HTTPS 인증서 문제를 우회하기 위해 추가

# 🚨 중요: 본인의 Naver API ID와 Secret Key를 입력하세요.
client_id = 'Kc65Ibtu71R5TLRUF7lL'
client_secret = 'CdT4sLf9Ht' # ⬅️ 여기에 본인의 Secret Key를 입력하세요.

# ===================================================================
# 2. 기본 함수 정의 (이전 코드와 유사)
# API URL에 요청을 보내고 응답을 받아오는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    # --- 추가된 부분: 웹 브라우저인 척 하기 위한 헤더 ---
    req.add_header("User-Agent", "Mozilla/5.0")

    try:
        # --- 추가된 부분: HTTPS 인증서 검증 비활성화 (일부 사이트의 인증서 오류 방지) ---
        context = ssl._create_unverified_context()
        response = urllib.request.urlopen(req, context=context)

        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print(f"[{datetime.datetime.now()}] Error for URL : {url}")
        return None

# 네이버 이미지 검색 API를 호출하는 함수
def getNaverImageSearch(srcText, start, display):
    base = "https://openapi.naver.com/v1/search/image" # 이미지 검색 엔드포인트
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}"
    url = base + parameters
    responseDecode = getRequestUrl(url)
    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

# ===================================================================
# 3. 이미지 크롤링 실행 (메인 로직)
# ===================================================================
# 검색할 키워드 목록
KEYWORDS = ["뿌팟퐁커리", "팟타이", "분짜"]
# 각 키워드당 다운로드할 이미지 개수
NUM_IMAGES = 100
# 이미지를 저장할 최상위 폴더 이름
SAVE_DIR = "naver_crawled_images1"

print("🚀 Naver 이미지 크롤링을 시작합니다...")

for keyword in KEYWORDS:
    print(f"\n[INFO] '{keyword}' 이미지 다운로드를 시작합니다...")

    download_path = os.path.join(SAVE_DIR, keyword)
    os.makedirs(download_path, exist_ok=True)

    downloaded_count = 0
    start_index = 1
    display_size = 100

    while downloaded_count < NUM_IMAGES:
        print(f"   -> API 요청: 시작 위치 {start_index}, 요청 개수 {display_size}")

        jsonResponse = getNaverImageSearch(keyword, start_index, display_size)

        if jsonResponse is None or 'items' not in jsonResponse or not jsonResponse['items']:
            print("   -> 더 이상 검색 결과가 없습니다. 다운로드를 중단합니다.")
            break

        for item in jsonResponse['items']:
            if downloaded_count >= NUM_IMAGES:
                break

            try:
                image_url = item['link']

                # --- 추가된 부분: 다운로드를 위한 별도의 요청 객체 생성 ---
                # 이 요청 객체에도 User-Agent 헤더를 추가하여 차단을 피합니다.
                opener = urllib.request.build_opener()
                opener.addheaders = [('User-agent', 'Mozilla/5.0')]
                urllib.request.install_opener(opener)

                file_ext = os.path.splitext(image_url)[1] or '.jpg' # 확장자가 없으면 .jpg로
                filename = f"{keyword}_{downloaded_count + 1}{file_ext}"
                save_path = os.path.join(download_path, filename)

                # 수정된 방식으로 이미지 다운로드
                urllib.request.urlretrieve(image_url, save_path)
                downloaded_count += 1

            except Exception as e:
                # 실패 로그를 좀 더 상세하게 출력
                print(f"   -> 다운로드 실패: {image_url} | 오류: {type(e).__name__} - {e}")

        start_index += display_size
        time.sleep(0.2) # API 요청 간격을 약간 늘려 서버 부담 감소

    print(f"✅ '{keyword}' 이미지 {downloaded_count}장 다운로드 완료! -> 저장 경로: {download_path}")

print("\n🎉 모든 Naver 이미지 크롤링 작업이 완료되었습니다.")

🚀 Naver 이미지 크롤링을 시작합니다...

[INFO] '뿌팟퐁커리' 이미지 다운로드를 시작합니다...
   -> API 요청: 시작 위치 1, 요청 개수 100
   -> 다운로드 실패: https://homecuisine.co.kr/files/attach/images/142/034/048/d65437be8115189cf060cf8c01e06ec6.JPG | 오류: URLError - <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>
   -> 다운로드 실패: http://t1.daumcdn.net/tvpot/thumb/v11d15RXjwJ5k8D6KKJCw56/thumb.png?time=1517363631242 | 오류: OSError - [Errno 22] Invalid argument: 'naver_crawled_images\\뿌팟퐁커리\\뿌팟퐁커리_9.png?time=1517363631242'
   -> 다운로드 실패: https://timg.humoruniv.com/thumb_crop_resize.php?url=https://down.humoruniv.com//hwiparambbs/data/editor/cook/e_s7fdd79012_10c28ead9fab23268c0a5a112692217f68375b18.jpg?SIZE=800x800 | 오류: OSError - [Errno 22] Invalid argument: 'naver_crawled_images\\뿌팟퐁커리\\뿌팟퐁커리_13.jpg?SIZE=800x800'
   -> 다운로드 실패: https://static.wtable.co.kr/image/production/upload/118100582/styles/15908/0c7a6070-fa2f-44ef-ba47-f4319a502045.jpg?size=800x800 | 오류: OSError - [Errno 22] Invalid argum